In [1]:
import pandas as pd
import altair as alt
import warnings
warnings.filterwarnings("ignore")
alt.renderers.set_embed_options(theme='light')
pd.set_option("display.max_colwidth", 400)
pd.set_option("display.max_rows", 400)


Trend line of what yLUNA is being used for?  PRISM Farm, yLUNA Staking, LPing, or Nothing.

In [2]:
prism_addr = 'terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw'
xprism_addr = 'terra1042wzrwg2uk6jqxjm34ysqquyr9esdgm5qyswz'
yluna_addr = 'terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz'
pluna_addr = 'terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2'
pLuna_PRISM_Pair = 'terra1persuahr6f8fm6nyup0xjc7aveaur89nwgs5vs'
yLuna_PRISM_Pair = 'terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce'
xprism_PRISM_Pair = 'terra1czynvm64nslq2xxavzyrrhau09smvana003nrf'

In [3]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [4]:
def get_url(url):
    return pd.read_csv(url, index_col=0)

In [5]:
class GenericDataProvider():
    def __init__(self, claim, get_url, config , path_to_data='../data'):
        self.config = config
        self.claim = claim
        self.get_url = get_url
        self.path_to_data = path_to_data
    
    def load(self):
        self.data = {}
        for name, c in self.config.items():
            current_data = {}
            df = self.claim(c['hash'])
            df.columns = [c.lower() for c in df.columns]
            current_data['from_hash'] = df
            try:
                df = pd.read_csv(f'{self.path_to_data}/{c["file_name"]}', index_col=0)
                df.columns = [c.lower() for c in df.columns]
                current_data['from_file'] = df
            except Exception as e:
                print(e)
            self.data[name] = current_data
        
    def standard_parse(self, df):
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x)
        df['hr'] = df.block_timestamp.str[:-5] + '00:00.000'
        df['day'] = df.block_timestamp.str[:-9]
        return df
        

In [6]:
swaps_config = {
     'swaps':{
                  'hash':'1bfd8019-89a1-470d-8868-60d71e57d1d0',
                  'url':'',
                  'file_name':'swaps.csv'},
     'router':{
                  'hash':'69b149fb-81ba-4860-aac1-b17f0d6d7688',
                  'url':'',
                  'file_name':'router.csv'}
}

In [7]:
class SwapsDataProvider(GenericDataProvider):
    def load(self):
        super().load()
        self.swaps_df = self.data['swaps']['from_hash']
        self.swaps_df_from_csv = self.data['swaps']['from_file']
        self.swaps_df = self.swaps_df.append(self.swaps_df_from_csv).drop_duplicates(ignore_index=True)
        self.swaps_df = self.standard_parse(self.swaps_df)
        #
        self.router_df = self.data['router']['from_hash']
        self.router_df_from_csv = self.data['router']['from_file']
        self.router_df = self.router_df.append(self.router_df_from_csv).drop_duplicates(ignore_index=True)
        self.router_df = self.standard_parse(self.router_df)
            
    def write_to_csv(self):
        self.swaps_df.to_csv(f'{self.path_to_data}/swaps.csv')
        self.router_df.to_csv(f'{self.path_to_data}/router.csv')
        
    def parse_simple_swaps(self):
        self.swaps_df.columns = [c.lower() for c in self.swaps_df.columns]
        swaps_df = self.swaps_df[self.swaps_df.ask_asset != '']
        swaps_df = swaps_df[swaps_df.sender.notna()]
        swaps_df_pol = swaps_df.rename(columns={'sender':'user','ask_asset':'asset_received','offer_asset':'asset_given'})
        swaps_df_pol = swaps_df_pol[['block_timestamp','tx_id','price','user','asset_received','return_amount','asset_given','offer_amount']]
        swaps_df_pol['operation'] = 'swap'
        self.swaps_df_pol = swaps_df_pol
        
    def parse_router(self):
        self.router_df.columns = [c.lower() for c in self.router_df.columns]
        router_df = self.router_df.rename(columns={'sender':'user'})
        router_df_1 = router_df[['block_timestamp','tx_id','0_ask_asset','0_offer_amount','0_offer_asset','0_price','0_return_amount','user']]
        router_df_1 = router_df_1.rename(columns={'0_ask_asset':'asset_received','0_offer_amount':'offer_amount',
                           '0_price':'price','0_return_amount':'return_amount','0_offer_asset':'asset_given'})
        router_df_2 = router_df[['block_timestamp','tx_id','1_ask_asset','1_offer_amount','1_offer_asset','1_price','1_return_amount','user']]
        router_df_2 = router_df_2.rename(columns={'1_ask_asset':'asset_received','1_offer_amount':'offer_amount',
                           '1_price':'price','1_return_amount':'return_amount','1_offer_asset':'asset_given'})
        router_df_pol = router_df_1.append(router_df_2)
        router_df_pol['operation'] = 'swap'
        self.router_df_pol = router_df_pol
    
    def parse(self):
        self.parse_simple_swaps()
        self.parse_router()
        self.swaps_df_all = self.router_df_pol.append(self.swaps_df_pol[self.router_df_pol.columns])
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x)
        self.swaps_df_all['hr'] = self.swaps_df_all.block_timestamp.str[:-5] + '00:00.000'
        self.swaps_df_all['day'] = self.swaps_df_all.block_timestamp.str[:-9]
        #
        yluna_swaps = self.swaps_df_all[(self.swaps_df_all.asset_given=='yLUNA')|(self.swaps_df_all.asset_received=='yLUNA')]
        yluna_swaps['amount_signed'] = yluna_swaps.apply(lambda row: row.offer_amount if row.asset_given == 'yLUNA' else -row.return_amount,axis=1)
        yluna_swaps['type'] = 'swap'
        self.yluna_swaps = yluna_swaps.drop_duplicates(['tx_id'],ignore_index=True)
        #
        pluna_swaps = self.swaps_df_all[(self.swaps_df_all.asset_given=='pLUNA')|(self.swaps_df_all.asset_received=='pLUNA')]
        pluna_swaps['amount_signed'] = pluna_swaps.apply(lambda row: row.offer_amount if row.asset_given == 'pLUNA' else -row.return_amount,axis=1)
        pluna_swaps['type'] = 'swap'
        self.pluna_swaps = pluna_swaps.drop_duplicates(['tx_id'],ignore_index=True)
        #
        given = swaps_dp.swaps_df_all.copy()
        given['asset'] = given.asset_given
        given['amount_signed'] = -given.offer_amount

        received = swaps_dp.swaps_df_all.copy()
        received['asset'] = received.asset_received
        received['amount_signed'] = received.return_amount

        cols = ['block_timestamp', 'tx_id','user', 'hr',
               'day', 'asset', 'amount_signed']
        swaps_single = received[cols].append(given[cols])
        self.swaps_single = swaps_single[swaps_single.amount_signed.notna()].drop_duplicates(['tx_id','asset'],ignore_index=True)
        

In [8]:
 config = {
     'transfers_prism': {'hash':'9651b671-1f85-4133-8384-20042dc1adb1',
                  'url':'',
                  'file_name':'transfers.csv'},
     'transfers_xprism' : {
                  'hash':'483df27b-4c69-47ec-8797-332e6c75ae86',
                  'url':'',
                  'file_name':'transfers_xprism.csv'},
     'boost_activation': {'hash':'ccbaf92f-6aaa-4460-a65f-83814431fd17',
                  'url':'',
                  'file_name':'boost_activation.csv'},
     'prism_forge': {
                  'hash':'2a364c52-9c37-4540-81c7-e5b890f0ef6b',
                  'url':'',
                  'file_name':'prism_forge.csv'},
     'amps_bound_unbound':{
                  'hash':'160d6d2c-487b-4173-af2e-ff51f9b64c0f',
                  'url':'',
                  'file_name':'amps_bound_unbound.csv'},
     'prism_stake':{
                  'hash':'7403d8c8-1687-44d3-9090-d72eec1407bc',
                  'url':'',
                  'file_name':'prism_stake.csv'}
}

In [9]:
class PrismBalanceDataProvider(GenericDataProvider):
    def load(self):
        super().load()
        self.transfers_df = self.data['transfers_prism']['from_hash']
        self.transfers_df_from_csv = self.data['transfers_prism']['from_file']
        self.transfers_df = self.transfers_df.append(self.transfers_df_from_csv).drop_duplicates(ignore_index=True)
        #
        self.transfers_xprism_df = self.data['transfers_xprism']['from_hash']
        self.transfers_xprism_df_from_csv = self.data['transfers_xprism']['from_file']
        self.transfers_xprism_df = self.transfers_xprism_df.append(self.transfers_xprism_df_from_csv).drop_duplicates(ignore_index=True)
        #
        self.boost_activation_df = self.data['boost_activation']['from_hash']
        self.boost_activation_df_from_csv = self.data['boost_activation']['from_file']
        self.boost_activation_df = self.boost_activation_df.append(self.boost_activation_df_from_csv).drop_duplicates(ignore_index=True)
        #self.boost_activation_df = self.standard_parse(self.boost_activation_df)
        #
        self.prism_forge_df = self.data['prism_forge']['from_hash']
        self.prism_forge_df_from_csv = self.data['prism_forge']['from_file']
        self.prism_forge_df = self.prism_forge_df.append(self.prism_forge_df_from_csv).drop_duplicates(ignore_index=True)
        #
        self.amps_bound_unbound_df = self.data['amps_bound_unbound']['from_hash']
        self.amps_bound_unbound_df_from_csv = self.data['amps_bound_unbound']['from_file']
        self.amps_bound_unbound_df = self.amps_bound_unbound_df.append(self.amps_bound_unbound_df_from_csv).drop_duplicates(ignore_index=True)
        #
        self.prism_stake_df = self.data['prism_stake']['from_hash']
        self.prism_stake_df_from_csv = self.data['prism_stake']['from_file']
        self.prism_stake_df = self.prism_stake_df.append(self.prism_stake_df_from_csv).drop_duplicates(ignore_index=True)
        
    def parse_forge_withdrawals(self):
        self.prism_forge_df = self.standard_parse(self.prism_forge_df)
        prism_forge_df = self.prism_forge_df
        prism_forge_df['amount'] = prism_forge_df.withdraw_amount/1000000
        prism_forge_df = prism_forge_df[['block_timestamp','tx_id','to_','amount','day']]
        prism_forge_df.columns = ['block_timestamp','tx_id','user','amount','day']
        prism_forge_df['amount_signed'] = prism_forge_df.amount
        self.prism_forge_df = prism_forge_df.drop_duplicates(ignore_index=True)
        
    def parse_transfers(self):
        self.transfers_df = self.standard_parse(self.transfers_df)
        self.transfers_df['amount'] = self.transfers_df.amount/1000000
        transfers_df = self.transfers_df
        send_tx = transfers_df[['block_timestamp','tx_id','from_','amount','day']]
        send_tx.columns = ['block_timestamp','tx_id','user','amount','day']
        send_tx['amount_signed'] = -send_tx.amount
        receive_tx = transfers_df[['block_timestamp','tx_id','to_','amount','day']]
        receive_tx.columns = ['block_timestamp','tx_id','user','amount','day']
        receive_tx['amount_signed'] = receive_tx.amount
        self.all_transfers = send_tx.append(receive_tx).drop_duplicates(ignore_index=True)
    
    def parse_transfers_xprism(self):
        self.transfers_xprism_df = self.standard_parse(self.transfers_xprism_df)
        self.transfers_xprism_df['amount'] = self.transfers_xprism_df.amount/1000000
        transfers_df = self.transfers_xprism_df
        send_tx = transfers_df[['block_timestamp','tx_id','from_','amount','day']]
        send_tx.columns = ['block_timestamp','tx_id','user','amount','day']
        send_tx['amount_signed'] = -send_tx.amount
        receive_tx = transfers_df[['block_timestamp','tx_id','to_','amount','day']]
        receive_tx.columns = ['block_timestamp','tx_id','user','amount','day']
        receive_tx['amount_signed'] = receive_tx.amount
        self.all_transfers_xprism = send_tx.append(receive_tx).drop_duplicates(ignore_index=True)
        
    def parse_amps_bonding(self):
        self.amps_bound_unbound_df = self.standard_parse(self.amps_bound_unbound_df)
        amps_bound_unbound_df = self.amps_bound_unbound_df
        amps_bound_unbound_df['amount'] = amps_bound_unbound_df['amount']/1000000
        amps_bound_unbound_df['action'] = amps_bound_unbound_df.apply(lambda x: 'unbond' if ((x['action']=='transfer')or(x['1_action']=='transfer')or(x['0_action']=='transfer')) else 'bond',axis=1)
        amps_bound_unbound_df['amount_signed'] = amps_bound_unbound_df.apply(lambda x: x.amount if x['action']=='unbond' else -x.amount,axis=1)
        amps_bound_unbound_df['user'] = amps_bound_unbound_df.apply(lambda x: x.to_ if x['action']=='unbond' else x.from_,axis=1)
        self.amps_bound_unbound_df = amps_bound_unbound_df[['block_timestamp', 'user', 'from_', 'hr',
               'msg_index', 'to_', 'tx_id', 'action', 'amount', 'day',
               'amount_signed']].drop_duplicates(ignore_index=True)
        
    def get_xprism_stake_amount(self, row):
        if(row['1_action']=='redeem_xprism'):
            return -row['amount']/1000000
        if(row['1_action']=='mint_xprism'):
            return row['1_amount']/1000000
    def get_prism_stake_amount(self, row):
        if(row['1_action']=='redeem_xprism'):
            return row['prism_queued']/1000000
        if(row['1_action']=='mint_xprism'):
            return -row['0_amount']/1000000
    
    def parse_prism_staking(self):
        self.prism_stake_df = self.standard_parse(self.prism_stake_df)
        prism_stake_df = self.prism_stake_df
        prism_stake_df['xprism_amount_signed'] = prism_stake_df.apply(self.get_xprism_stake_amount,axis=1)
        prism_stake_df['prism_amount_signed'] = prism_stake_df.apply(self.get_prism_stake_amount,axis=1)
        prism_stake_df['user'] = prism_stake_df['from_']
        prism_stake_df['action'] = prism_stake_df['1_action']
        self.prism_stake_df = prism_stake_df[['block_timestamp', 'hr', 'user',
               'tx_id', 'action', 'amount', 'prism_queued', 'day',
               'xprism_amount_signed', 'prism_amount_signed']].drop_duplicates(ignore_index=True)

    def write_to_csv(self):
        self.transfers_df.to_csv(f'{self.path_to_data}/transfers.csv')
        self.transfers_xprism_df.to_csv(f'{self.path_to_data}/transfers_xprism.csv')
        self.boost_activation_df.to_csv(f'{self.path_to_data}/boost_activation.csv')
        self.prism_forge_df.to_csv(f'{self.path_to_data}/prism_forge.csv')
        self.amps_bound_unbound_df.to_csv(f'{self.path_to_data}/amps_bound_unbound.csv')
        self.prism_stake_df.to_csv(f'{self.path_to_data}/prism_stake.csv')
        
    def parse(self):
        self.parse_transfers()
        self.parse_transfers_xprism()
        self.parse_forge_withdrawals()
        self.parse_amps_bonding()
        self.parse_prism_staking()
            
    

In [10]:
lp_config = {
     'lp':{
                  'hash':'eaf604bc-9cf6-4590-8540-0bb1aafd9d7e',
                  'url':'',
                  'file_name':'lp_provide_withdraw.csv'}
}

In [11]:
class LPDataProvider(GenericDataProvider):
    def load(self):
        super().load()
        self.lp_provide_withdraw_df = self.data['lp']['from_hash']
        self.lp_provide_withdraw_df_from_csv = self.data['lp']['from_file']
        self.lp_provide_withdraw_df = self.lp_provide_withdraw_df.append(self.lp_provide_withdraw_df_from_csv).drop_duplicates(ignore_index=True)
            
    def write_to_csv(self):
        self.lp_provide_withdraw_df.to_csv(f'{self.path_to_data}/lp_provide_withdraw.csv')
        
    def get_action(self, row):
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(f'{prefix}action' in row):
                if(row[f'{prefix}action'] in ['provide_liquidity','withdraw_liquidity']):
                    return row[f'{prefix}action']
                
    def get_n_action(self, row):
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(f'{prefix}action' in row):
                if(row[f'{prefix}action'] in ['provide_liquidity','withdraw_liquidity']):
                    return f'{prefix}'
                
    def correct_parsing(self, row):
        if (row.prefix == '1_' and row.f_action == 'withdraw_liquidity') or \
            (row.prefix == '0_' and row.f_action == 'provide_liquidity'):
            return False
        else:
            return True
                
    def parse(self):
        self.lp_provide_withdraw_df = self.standard_parse(self.lp_provide_withdraw_df)
        df = self.lp_provide_withdraw_df
        df['f_action'] = df.apply(self.get_action,axis=1)
        df['prefix'] = df.apply(self.get_n_action,axis=1)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x) 
        df['f_contract_address'] = df.apply(lambda row: row[f'{row.prefix}contract_address'], axis=1)
        df['asset'] = df.apply(lambda row: 'xPRISM' if row.f_contract_address==xprism_PRISM_Pair \
                                    else '', axis=1)
        assert df.apply(self.correct_parsing , axis=1).sum() == 0
        provide_ = df[df.f_action=='provide_liquidity']
        withdraw_ = df[df.f_action=='withdraw_liquidity']
        #
        provide_['prism_amount'] = provide_.apply(lambda row: row['2_amount'] if row['1_contract_address'] == prism_addr else row['3_amount'],axis=1)
        provide_['asset_amount'] = provide_.apply(lambda row: row['1_amount'] if row['2_contract_address'] in [xprism_addr] else row['2_amount'],axis=1)
        provide_['user'] = provide_['from_']
        provide_['hr'] = provide_.block_timestamp.str[:-5] + '00:00.000'
        provide_['day'] = provide_.block_timestamp.str[:-9]
        provide_ = provide_[['block_timestamp','user','tx_id','f_action','prism_amount','asset','asset_amount','hr','day']]
        provide_['amount_signed'] = -provide_.asset_amount
        provide_['type'] = 'provide_lp'
        self.provide_ = provide_.drop_duplicates(ignore_index=True)
        #
        withdraw_['prism_amount'] = withdraw_.apply(lambda row: row['1_amount'] if row['2_contract_address'] == prism_addr else row['2_amount'],axis=1)
        withdraw_['asset_amount'] = withdraw_.apply(lambda row: row['2_amount'] if row['3_contract_address'] in [xprism_addr] else row['1_amount'],axis=1)
        withdraw_['user'] = withdraw_['from_']
        withdraw_['hr'] = withdraw_.block_timestamp.str[:-5] + '00:00.000'
        withdraw_['day'] = withdraw_.block_timestamp.str[:-9]
        withdraw_ = withdraw_[['block_timestamp','user','tx_id','f_action','prism_amount','asset','asset_amount','hr','day']]
        withdraw_['amount_signed'] = withdraw_.asset_amount
        withdraw_['type'] = 'withdraw_lp'
        self.withdraw_ = withdraw_.drop_duplicates(ignore_index=True)
        self.all_lps = withdraw_[['day','user','asset','amount_signed','tx_id','type','block_timestamp']]\
                            .append(provide_[['day','user','asset','amount_signed','tx_id','type','block_timestamp']]).drop_duplicates(ignore_index=True)
                            

In [12]:
lp_dp = LPDataProvider(claim, get_url, lp_config, '../data/prism')
lp_dp.load()
lp_dp.write_to_csv()
lp_dp.parse()

In [13]:
swaps_dp = SwapsDataProvider(claim, get_url, swaps_config, '../data/')
swaps_dp.load()
swaps_dp.write_to_csv()
swaps_dp.parse()

In [14]:
prism_dp = PrismBalanceDataProvider(claim, get_url, config, '../data/prism')
prism_dp.load()
prism_dp.write_to_csv()
prism_dp.parse()

**Forge into staking**

In [15]:
forge_participants = pd.DataFrame(prism_dp.prism_forge_df.user.unique(),columns=['forge_participant'])
stake_participants = pd.DataFrame(prism_dp.prism_stake_df.user.unique(),columns=['stake_participant'])
merge = forge_participants.merge(stake_participants, 
                                 left_on='forge_participant', 
                                 right_on='stake_participant', 
                                 how='left')
df = merge.stake_participant.fillna(0).apply(lambda x: 'Forge & Stake' if x==0 else 'Only Forge').value_counts()
df=df.reset_index().rename(columns={'index':'Type','stake_participant':'Number of users'})

In [16]:
pie_ust_chart = alt.Chart(df).mark_arc(innerRadius=60).encode(
    theta=alt.Theta(field="Number of users", type="quantitative"),
    color=alt.Color(field="Type", type="nominal",
            sort=['Number of users'],
            scale=alt.Scale(scheme='pastel1'),
            legend=alt.Legend(
            orient='none',
            padding=10,
            legendY=-10,
            direction='vertical')),
    tooltip=['Type','Number of users']
).configure_view(strokeOpacity=0)
pie_ust_chart

alt.Chart(...)

**xPRISM Balances**

In [17]:
df = lp_dp.all_lps
lp_xprism_bal = df.groupby(['user','day','asset']).amount_signed.sum().reset_index()

In [18]:
df = prism_dp.all_transfers_xprism
df['asset'] = 'xPRISM'
transfer_xprism_bal = df.groupby(['user','day','asset']).amount_signed.sum().reset_index()

In [19]:
df = prism_dp.all_transfers
df['asset'] = 'PRISM'
transfer_bal = df.groupby(['user','day','asset']).amount_signed.sum().reset_index()

In [20]:
df = prism_dp.amps_bound_unbound_df
df['amount_signed'] = -df['amount_signed']
df['asset'] = 'xPRISM in AMPs'
amps_bal = df.groupby(['user','day','asset']).amount_signed.sum().reset_index()

In [21]:
df = prism_dp.prism_stake_df
df_xprism = df.copy()
df_xprism['amount_signed'] = df_xprism['xprism_amount_signed']
df_xprism['asset'] = 'xPRISM'
df_xprism[['block_timestamp', 'hr', 'user', 'tx_id', 'action','asset','amount_signed']]
df_prism = df.copy()
df_prism['amount_signed'] = df_prism['prism_amount_signed']
df_prism['asset'] = 'PRISM'
stake_bal = df_prism.append(df_xprism).groupby(['user','day','asset']).amount_signed.sum().reset_index()

In [22]:
stake_bal[stake_bal.user=='terra1y5c7zwhlagjvk04mqd54rdw3sdk83hvxjjf8d3']

,user,day,asset,amount_signed
15440,terra1y5c7zwhlagjvk04mqd54rdw3sdk83hvxjjf8d3,2022-02-06,PRISM,-655000.000000
15441,terra1y5c7zwhlagjvk04mqd54rdw3sdk83hvxjjf8d3,2022-02-06,xPRISM,654700.246844
15442,terra1y5c7zwhlagjvk04mqd54rdw3sdk83hvxjjf8d3,2022-02-08,PRISM,-1245.000000
15443,terra1y5c7zwhlagjvk04mqd54rdw3sdk83hvxjjf8d3,2022-02-08,xPRISM,1242.969498


In [23]:
df = prism_dp.prism_forge_df
df = df.groupby(['user','day']).amount_signed.sum().reset_index()
df['asset'] = 'PRISM'
forge_bal = df

In [24]:
swaps_bal = swaps_dp.swaps_single\
                .groupby(['user','day','asset']).amount_signed.sum().reset_index()

In [25]:
balances = swaps_bal.append(forge_bal).append(stake_bal).append(amps_bal)\
                    .append(transfer_bal).append(transfer_xprism_bal).append(lp_xprism_bal)
balances=balances.groupby(['user','asset']).amount_signed.sum().reset_index()

balances_cumsum = balances.pivot(columns='asset',values='amount_signed',index=['day','user'])\
                          .fillna(0).groupby(level=-1).cumsum().reset_index()

In [26]:
balances_pivoted = balances.pivot(columns='asset',values='amount_signed',index=['user'])\
                          .fillna(0).reset_index()

In [27]:
balances_pivoted[balances_pivoted.user\
                .isin(['terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv','terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q'])]\
                .sort_values(by=['user'])


asset,user,LUNA,PRISM,UST,cLUNA,pLUNA,xPRISM,xPRISM in AMPs,yLUNA
534,terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv,0.000000,-5.684342e-14,0.0,0.000000,0.000000,1474.884278,1474.884278,0.0
9467,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,-22.524349,5.068228e+03,-65.0,21.577678,2.666191,137.777889,147.341291,0.0


In [28]:
balances_pivoted = balances_pivoted[balances_pivoted['xPRISM']>0]
balances_pivoted['perc_in_amps'] = balances_pivoted['xPRISM in AMPs']/balances_pivoted['xPRISM']
def dd(x):
    if(x<0):
        return 0
    if(x>1):
        return 1
    return x
def dd_range(x):
    return int((x*100-1)/10)
balances_pivoted['perc_in_amps'] = balances_pivoted['perc_in_amps'].apply(dd)
df = balances_pivoted['perc_in_amps'].apply(dd_range).value_counts()

In [29]:
balances_pivoted[balances_pivoted.user\
                .isin(['terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv','terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q'])]\
                .sort_values(by=['user'])

asset,user,LUNA,PRISM,UST,cLUNA,pLUNA,xPRISM,xPRISM in AMPs,yLUNA,perc_in_amps
534,terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv,0.000000,-5.684342e-14,0.0,0.000000,0.000000,1474.884278,1474.884278,0.0,1.0
9467,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,-22.524349,5.068228e+03,-65.0,21.577678,2.666191,137.777889,147.341291,0.0,1.0


In [30]:
df = df.reset_index()
df['perc_in_amps'] = df['perc_in_amps']/df['perc_in_amps'].sum()*100
df.columns = ['Percentage of xPRISM in AMPs', '% of users']
df['% of users'] = df['% of users'].apply(lambda x: round(x,2))

In [31]:
m = {}
for i in range(10):
    m[i]=f"{(i)*10}-{(i+1)*10}"
df['Percentage of xPRISM in AMPs'] = df['Percentage of xPRISM in AMPs'].map(m)

In [32]:
alt.Chart(df).mark_bar().encode(
    x = alt.X('Percentage of xPRISM in AMPs:N', axis=alt.Axis(tickCount=10, labelAngle=0, tickBand = 'center')),
    y='% of users',
    tooltip= ['Percentage of xPRISM in AMPs', '% of users']
).properties(width=600)

alt.Chart(...)

**Prism stake**

In [33]:
prism_dp.prism_stake_df[prism_dp.prism_stake_df.user.isin(['terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv','terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q'])]


,block_timestamp,hr,user,tx_id,action,amount,prism_queued,day,xprism_amount_signed,prism_amount_signed


**AMPs**

In [34]:
prism_dp.amps_bound_unbound_df[prism_dp.amps_bound_unbound_df.user.isin(['terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv','terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q'])]


,block_timestamp,user,from_,hr,msg_index,to_,tx_id,action,amount,day,amount_signed,asset
3761,2022-02-23 18:45:34,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,2022-02-23 18:00:00.000,0,terra1pa4amk66q8punljptzmmftf6ylq3ezyzx6kl9m,E4907CD80E944669F81E3CCAF29059CB53C3AFCBC5742C8A5B5AC087D0AF5EE6,bond,137.777889,2022-02-23,137.777889,xPRISM in AMPs
7192,2022-02-28 18:35:13,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,2022-02-28 18:00:00.000,0,terra1pa4amk66q8punljptzmmftf6ylq3ezyzx6kl9m,33E962D4919B200AB41F7D2AC28EAAF4C3A6C1F1D7E279D41A1DEB64BBC35234,bond,9.563402,2022-02-28,9.563402,xPRISM in AMPs
7219,2022-02-23 18:41:46,terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv,terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv,2022-02-23 18:00:00.000,0,terra1pa4amk66q8punljptzmmftf6ylq3ezyzx6kl9m,A5304009F2252072CC888E8136C5E029D5E410A0285AA77E573400422E144047,bond,1474.884278,2022-02-23,1474.884278,xPRISM in AMPs


**xPRISM transfers**

In [35]:
prism_dp.all_transfers_xprism[prism_dp.all_transfers_xprism.user=='terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q']

,block_timestamp,tx_id,user,amount,day,amount_signed,asset
42,2022-02-15 19:33:01,8CF7468DE489FFBEB2397A6BA58F746A623FAFF3C0131CA19C27BC1B6A9A9A25,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,566.82875,2022-02-15,-566.82875,xPRISM


In [36]:
prism_dp.all_transfers_xprism[prism_dp.all_transfers_xprism.user=='terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv']

,block_timestamp,tx_id,user,amount,day,amount_signed,asset
153,2022-02-15 19:33:01,8CF7468DE489FFBEB2397A6BA58F746A623FAFF3C0131CA19C27BC1B6A9A9A25,terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv,566.82875,2022-02-15,566.82875,xPRISM
